In [1]:
!pip install tensorflow_datasets

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import tensorflow as tf

from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)
#tf.config.experimental.set_memory_growth(physical_devices[1], True)


#used to fix bug in keras preprocessing scope
temp = tf.zeros([4, 32, 32, 3])  # Or tf.zeros
preprocess_input(temp)
print("processed")

processed


In [3]:
IMAGE_SIZE = (224, 224)
TRAIN_SIZE = 50000
VALIDATION_SIZE = 10000
BATCH_SIZE_PER_GPU = 16
global_batch_size = (BATCH_SIZE_PER_GPU * 1)
NUM_CLASSES = 10

Dataset code

In [4]:
def flip(x: tf.Tensor) -> tf.Tensor:
    """Flip augmentation

    Args:
        x: Image to flip

    Returns:
        Augmented image
    """
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)

    return x

def color(x: tf.Tensor) -> tf.Tensor:
    """Color augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x

def rotate(x: tf.Tensor) -> tf.Tensor:
    """Rotation augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    return tf.image.rot90(x, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

def zoom(x: tf.Tensor) -> tf.Tensor:
    """Zoom augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    # Generate 20 crop settings, ranging from a 1% to 20% crop.
    scales = list(np.arange(0.8, 1.0, 0.01))
    boxes = np.zeros((len(scales), 4))

    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(img):
        # Create different crops for an image
        crops = tf.image.crop_and_resize([img], boxes=boxes, box_indices=np.zeros(len(scales)), crop_size=IMAGE_SIZE)
        # Return a random crop
        return crops[tf.random.uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]


    choice = tf.random.uniform(())

    # Only apply cropping 50% of the time
    return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))

def normalize(input_image):
  return preprocess_input(input_image)

@tf.function
def load_image_train(datapoint):
  input_image, label = tf.image.resize(datapoint["image"], IMAGE_SIZE), datapoint['label']
  # if tf.random.uniform(()) > 0.5:
  #   input_image = tf.image.flip_left_right(input_image)
  augmentations = [flip, color, zoom, rotate]
  for f in augmentations:
    input_image = tf.cond(tf.random.uniform(()) > 0.75, lambda: f(input_image), lambda: input_image)

  #input_image = preprocess_input(input_image)
  input_image = normalize(input_image)

  return input_image, tf.one_hot(label, depth=NUM_CLASSES)

@tf.function
def load_image_test(datapoint):
  input_image, label = tf.image.resize(datapoint["image"], IMAGE_SIZE), datapoint['label']
  #input_image = preprocess_input(input_image)

  input_image = normalize(input_image)

  return input_image, tf.one_hot(label, depth=NUM_CLASSES)

In [5]:
dataset, info = tfds.load('cifar10', with_info=True)

In [6]:
for example in dataset['train'].take(1):
  print(example["label"])

tf.Tensor(3, shape=(), dtype=int64)


In [7]:
info

tfds.core.DatasetInfo(
    name='cifar10',
    version=1.0.2,
    description='The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.',
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    features=FeaturesDict({
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=60000,
    splits={
        'test': 10000,
        'train': 50000,
    },
    supervised_keys=('image', 'label'),
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
        title = {Learning multiple layers of features from tiny images},
        institution = {},
        year = {2009}
    }""",
    redistribution_info=,
)

make the upscaled cifar dataset

In [8]:
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train.shuffle(buffer_size=1000).batch(global_batch_size).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


test_dataset = dataset['test'].map(load_image_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(global_batch_size).repeat()
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [9]:
model = tf.keras.models.load_model('./base_model_cifar10_vgg16.h5')

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [11]:
model.compile(optimizer=tf.optimizers.SGD(learning_rate=.01, momentum=.9, nesterov=True), loss='mse', metrics=['acc'])

In [12]:
model.evaluate(test_dataset, steps=VALIDATION_SIZE//global_batch_size)

625/625 [==============================] - 36s 58ms/step - loss: 0.0091 - acc: 0.9399


[0.009067213316784818, 0.9399]

build training output

In [13]:
import math
class LayerBatch(tf.keras.utils.Sequence):
    
    def __init__(self, input_model, dataset):
        self.input_model = input_model
        self.dataset = dataset.__iter__()
        
    def __len__(self):
        return math.ceil(TRAIN_SIZE // global_batch_size )
    
    def __getitem__(self, index):
        X, y = self.input_model(next(self.dataset))
        return X, y
    
class LayerBatchSynth(tf.keras.utils.Sequence):
    
    def __init__(self, input_model, dataset):
        self.input_model = input_model
        self.dataset = dataset.__iter__()
        
    def __len__(self):
        return math.ceil(4224 // global_batch_size )
    
    def __getitem__(self, index):
        X, y = self.input_model(next(self.dataset))
        return X, y
    
import math
class LayerTest(tf.keras.utils.Sequence):
    
    def __init__(self, input_model, dataset):
        self.input_model = input_model
        self.dataset = dataset.__iter__()
        
    def __len__(self):
        return math.ceil(VALIDATION_SIZE // global_batch_size )
    
    def __getitem__(self, index):
        X, y = self.input_model(next(self.dataset))
        return X, y

In [14]:
def add_layers(inputs, filters, layers=2):
    print(inputs.get_shape())
    X = tf.keras.layers.SeparableConv2D(name=f'sep_conv_{build_replacement.counter}', filters=filters, 
                                        kernel_size= (3,3),
                                        padding='Same')(inputs)
    #X = tf.keras.layers.BatchNormalization(name=f'batch_norm_{build_replacement.counter}')(X)
    X = tf.keras.layers.ReLU(name=f'relu_{build_replacement.counter}')(X)
    
    build_replacement.counter += 1
    
    for i in range(1, layers):
        X = tf.keras.layers.SeparableConv2D(name=f'sep_conv_{build_replacement.counter}', filters=filters,
                                            kernel_size=(3,3), 
                                            padding='Same')(X)
        #X = tf.keras.layers.BatchNormalization(name=f'batch_norm_{build_replacement.counter}')(X)
        X = tf.keras.layers.ReLU(name=f'relu_{build_replacement.counter}')(X)
        build_replacement.counter += 1
    
    return X
    
def build_replacement(get_output, layers=2):
    inputs = tf.keras.Input(shape=get_output.output[0].shape[1::])
    
    X = add_layers(inputs, get_output.output[1].shape[-1], layers)
    replacement_layers = tf.keras.Model(inputs=inputs, outputs=X)
    return replacement_layers

build_replacement.counter = 0

In [15]:
def replac(inp, filters):
    
    return add_layers(inp, filters,layers=2)

In [16]:
import pprint
targets = []
for i, layer in enumerate(model.layers):
    if layer.__class__.__name__ == "Conv2D":
        if layer.kernel_size[0] == 3:
            #print(f'{i} layer {layer.name} , kernel size {layer.kernel_size}')
            targets.append({'name': layer.name, 'layer': i})

pprint.pprint(targets)

targets[0]['name'][:-4]

# add all layers to replace


[{'layer': 1, 'name': 'block1_conv1'},
 {'layer': 2, 'name': 'block1_conv2'},
 {'layer': 4, 'name': 'block2_conv1'},
 {'layer': 5, 'name': 'block2_conv2'},
 {'layer': 7, 'name': 'block3_conv1'},
 {'layer': 8, 'name': 'block3_conv2'},
 {'layer': 9, 'name': 'block3_conv3'},
 {'layer': 11, 'name': 'block4_conv1'},
 {'layer': 12, 'name': 'block4_conv2'},
 {'layer': 13, 'name': 'block4_conv3'},
 {'layer': 15, 'name': 'block5_conv1'},
 {'layer': 16, 'name': 'block5_conv2'},
 {'layer': 17, 'name': 'block5_conv3'}]


'block1_c'

In [ ]:
#%%monitor_cell "layers_vgg"
for target in targets:
    
    print(f"training layer {target['name']}")
    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model('base_model_cifar10_vgg16.h5')
    in_layer = target['layer']
    get_output = tf.keras.Model(inputs=model.input, outputs=[model.layers[in_layer - 1].output, 
                                                             model.layers[in_layer].output])


    replacement_layers = build_replacement(get_output, layers=2)
    replacement_len = len(replacement_layers.layers)
    layer_train_gen = LayerBatch(get_output, train_dataset)
    layer_test_gen = LayerTest(get_output, test_dataset)




    MSE = tf.losses.MeanSquaredError()

    optimizer=tf.keras.optimizers.SGD(.00001, momentum=.9, nesterov=True)
    replacement_layers.compile(loss=MSE, optimizer=optimizer)

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(patience=5, min_lr=.0001, factor=.3, verbose=1)
    early_stop = tf.keras.callbacks.EarlyStopping(patience=15, min_delta=.0001, restore_best_weights=True, verbose=1)
    history = replacement_layers.fit(x=layer_train_gen,
                                   epochs=50,
                                   steps_per_epoch=TRAIN_SIZE // global_batch_size ,
                                   validation_data=layer_test_gen,
                                   shuffle=False,
                                   callbacks=[reduce_lr, early_stop],
                                   validation_steps=VALIDATION_SIZE // global_batch_size,
                                   verbose=1)
    target['weights'] = [replacement_layers.layers[1].get_weights(), replacement_layers.layers[3].get_weights()]

training layer block1_conv1
(None, None, None, 3)
  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 3125 steps, validate for 625 steps
Epoch 1/50
3125/3125 [==============================] - 859s 275ms/step - loss: 201.5025 - val_loss: 31.5815
Epoch 2/50
3125/3125 [==============================] - 860s 275ms/step - loss: 26.8766 - val_loss: 21.1656
Epoch 3/50
3125/3125 [==============================] - 860s 275ms/step - loss: 22.9465 - val_loss: 19.1049
Epoch 4/50
3125/3125 [==============================] - 860s 275ms/step - loss: 21.2953 - val_loss: 17.7146
Epoch 5/50
3125/3125 [==============================] - 860s 275ms/step - loss: 20.0541 - val_loss: 16.8129
Epoch 6/50
3125/3125 [==============================] - 860s 275ms/step - loss: 19.5021 - val_loss: 16.3312
Epoch 7/50
3125/3125 [==============================] - 861s 276ms/step - loss: 18.8426 - val_loss: 15.8513
Epoch 8/50
3125/3125 [==============================] - 863s 276ms/step - loss: 18.4856 - val_loss: 15.3827
Epoch 9/50
3125/3125 [==============================] - 861s 275ms/step - loss: 18.0298 - 

  ...
    to  
  ['...']


  ...
    to  
  ['...']


  ...
    to  
  ['...']


Train for 3125 steps, validate for 625 steps
Epoch 1/50
3125/3125 [==============================] - 1363s 436ms/step - loss: 817.9617 - val_loss: 196.1347
Epoch 2/50
3125/3125 [==============================] - 1363s 436ms/step - loss: 167.6087 - val_loss: 149.4658
Epoch 3/50
2826/3125 [==========================>...] - ETA: 1:59 - loss: 137.2146

In [21]:
for target in targets:
    print(target['name'])

block1_conv1
block1_conv2
block2_conv1
block2_conv2
block3_conv1
block3_conv2
block3_conv3
block4_conv1
block4_conv2
block4_conv3
block5_conv1
block5_conv2
block5_conv3


I found this code on stackoverflow but It has some problems even though I think it is mostly on the right track

In [22]:

def make_list(X):
    if isinstance(X, list):
        return X
    return [X]

def list_no_list(X):
    if len(X) == 1:
        return X[0]
    return X

def replace_layer(model, replace_layer_subname, replacement_fn,
**kwargs):
    """
    args:
        model :: keras.models.Model instance
        replace_layer_subname :: str -- if str in layer name, replace it
        replacement_fn :: fn to call to replace all instances
            > fn output must produce shape as the replaced layers input
    returns:
        new model with replaced layers
    quick examples:
        want to just remove all layers with 'batch_norm' in the name:
            > new_model = replace_layer(model, 'batch_norm', lambda **kwargs : (lambda u:u))
        want to replace all Conv1D(N, m, padding='same') with an LSTM (lets say all have 'conv1d' in name)
            > new_model = replace_layer(model, 'conv1d', lambda layer, **kwargs: LSTM(units=layer.filters, return_sequences=True)
    """
    model_inputs = []
    model_outputs = []
    tsr_dict = {}

    model_output_names = [out.name for out in make_list(model.output)]

    for i, layer in enumerate(model.layers):
        ### Loop if layer is used multiple times
        for j in range(len(layer._inbound_nodes)):

            ### check layer inp/outp
            inpt_names = [inp.name for inp in make_list(layer.get_input_at(j))]
            outp_names = [out.name for out in make_list(layer.get_output_at(j))]

            ### setup model inputs
            if 'input' in layer.name:
                for inpt_tsr in make_list(layer.get_output_at(j)):
                    model_inputs.append(inpt_tsr)
                    tsr_dict[inpt_tsr.name] = inpt_tsr
                continue

            ### setup layer inputs
            # I added the exception model_3_3/Identity:0 I think the problem is that is the input layer
            inpt = list_no_list([tsr_dict[name]  for name in inpt_names])

            ### remake layer 
            if layer.name in replace_layer_subname:
              if "relu" in layer.name or 'bn' in layer.name:
                print('deleting ' + layer.name)
                x = inpt
              else:
                print('replacing '+layer.name)
                x = replacement_fn(inpt)
            else:
                x = layer(inpt)

            ### reinstantialize outputs into dict
            for name, out_tsr in zip(outp_names, make_list(x)):

                ### check if is an output
                if name in model_output_names:
                    model_outputs.append(out_tsr)
                tsr_dict[name] = out_tsr

    return tf.keras.models.Model(model_inputs, model_outputs)

In [23]:
targets[0]['weights'][0][0].shape

(3, 3, 3, 1)

In [24]:
targets[0]['weights']

[[array([[[[-0.15600105],
           [-0.36613223],
           [-0.41866478]],
  
          [[ 0.4907311 ],
           [ 0.6770429 ],
           [ 0.6049432 ]],
  
          [[-0.323776  ],
           [-0.31277326],
           [-0.19798154]]],
  
  
         [[[ 0.37914273],
           [ 0.57764566],
           [ 0.78544146]],
  
          [[ 1.4126973 ],
           [ 1.6896753 ],
           [ 1.52619   ]],
  
          [[ 0.75108045],
           [ 0.7470105 ],
           [ 0.47723055]]],
  
  
         [[[-0.24076457],
           [-0.21783897],
           [-0.34837812]],
  
          [[ 0.6945811 ],
           [ 0.6534224 ],
           [ 0.5996945 ]],
  
          [[-0.44389728],
           [-0.43446624],
           [-0.27165893]]]], dtype=float32),
  array([[[[-0.47871804,  0.29645342, -0.20238389, -0.5788659 ,
             0.18899713, -0.21864866,  0.5207042 , -0.53235406,
            -0.21930452, -0.12273967, -0.22654006,  0.18831822,
             0.28687224, -0.01104517, -0.201639

In [25]:
model.layers[targets[0]['layer']].output.shape[-1]

64

I think the problem with this is I cannot just insert this model, I need ot add each of the replacement blocks layers one by one

In [26]:
for target in targets:
    
    print(f'Replacing Layer {target["name"]}')
    
    tf.keras.backend.clear_session()
    
    model = tf.keras.models.load_model('base_model_cifar10_vgg16.h5')
    
    layer_name = target['name']
    layer_pos = target['layer']
    filters = model.layers[layer_pos].output.shape[-1]
    
    
    new_model = replace_layer(model, layer_name, lambda x: replac(x, filters))
    new_model.layers[layer_pos].set_weights(target['weights'][0])
    new_model.layers[layer_pos + 2].set_weights(target['weights'][1])
    new_model.compile(optimizer=tf.keras.optimizers.SGD(.1), loss="categorical_crossentropy", metrics=['accuracy'])
    target['score'] = new_model.evaluate(test_dataset, steps=VALIDATION_SIZE // global_batch_size)
    

Replacing Layer block1_conv1
replacing block1_conv1
(None, None, None, 3)
625/625 [==============================] - 43s 69ms/step - loss: 0.1943 - accuracy: 0.9395
Replacing Layer block1_conv2
replacing block1_conv2
(None, None, None, 64)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



625/625 [==============================] - 47s 75ms/step - loss: 0.1960 - accuracy: 0.9395
Replacing Layer block2_conv1
replacing block2_conv1
(None, None, None, 64)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



625/625 [==============================] - 40s 64ms/step - loss: 0.1967 - accuracy: 0.9397
Replacing Layer block2_conv2
replacing block2_conv2
(None, None, None, 128)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



...

625/625 [==============================] - 37s 59ms/step - loss: 0.1970 - accuracy: 0.9393
Replacing Layer block4_conv2
replacing block4_conv2
(None, None, None, 512)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



625/625 [==============================] - 36s 58ms/step - loss: 0.1951 - accuracy: 0.9392
Replacing Layer block4_conv3
replacing block4_conv3
(None, None, None, 512)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float16 by default, call `tf.keras.backend.set_floatx('float16')`. To change just this layer, pass dtype='float16' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



KeyError: 'weights'

In [ ]:
new_model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
new_model.evaluate(test_dataset, steps=VALIDATION_SIZE // global_batch_size)

In [ ]:
new_model.evaluate(test_dataset, steps=VALIDATION_SIZE // global_batch_size)